In [1]:
# python 3 notebook for Programmer's Club 18 Sept 2020

# import the usual data science and general things

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import time, random, warnings
warnings.filterwarnings('ignore')

# import what we need from Scikit-Learn
from sklearn import neighbors, neural_network
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer

# import several cutting edge tree-based learners
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

In [2]:
#from astroquery.sdss import SDSS  # enables direct queries to the SDSS database
#this example was taken from https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions
# also presented by Tunde

#STAR_GALquery = """SELECT TOP 20000
#                p.psfMag_r, p.fiberMag_r, p.fiber2Mag_r, p.petroMag_r, 
#                p.deVMag_r, p.expMag_r, p.modelMag_r, p.cModelMag_r, 
#                s.class
#                FROM PhotoObjAll AS p JOIN specObjAll s ON s.bestobjid = p.objid
#                WHERE p.mode = 1 AND s.sciencePrimary = 1 AND p.clean = 1 AND s.class != 'QSO'
#                ORDER BY p.objid ASC"""

#stars_gals = SDSS.query_sql(STAR_GALquery)
#stars_gals["class"]=np.array(stars_gals["class"], dtype=np.str)

# the above code throws errors on my Python 2.7 and 3.6.8 installations, which I will have to fix later ...
# -- thanks to Tom Scott for running it and sending the results
df = pd.read_csv('stars_gals')

# summary of the data:
print('number of astronomical objects:',len(df))
print('columns:',df.columns.values)
print('unique classes:',df['class'].unique())

number of astronomical objects: 20000
columns: ['psfMag_r' 'fiberMag_r' 'fiber2Mag_r' 'petroMag_r' 'deVMag_r' 'expMag_r'
 'modelMag_r' 'cModelMag_r' 'class']
unique classes: ['GALAXY' 'STAR']


In [3]:
# preprocessing before we apply machine learning

# create binary target column from class name strings
df['Target'] = (df['class'] == 'GALAXY').astype(int)
# check it worked
df[['Target','class']]

# make list of columns to use as training features
cols = df.columns.values.tolist()
cols.remove('class')
cols.remove('Target')

# standardize / scale the features to be used for model training
# note: magnitudes and colours often don't need scaling since they are already logarithmic
scaler = StandardScaler()
scaler.fit(df[cols])
df[cols] = scaler.transform(df[cols])
# let's looks at the results
df.head()

,psfMag_r,fiberMag_r,fiber2Mag_r,petroMag_r,deVMag_r,expMag_r,modelMag_r,cModelMag_r,class,Target
0,-0.460980,-0.456968,-0.387287,-0.655817,-0.719171,-0.669434,-0.601619,-0.655157,GALAXY,1
1,-0.169385,-0.068947,-0.116434,0.200199,0.249467,0.147178,0.219833,0.219415,STAR,0
2,0.306565,0.193642,0.259301,0.367189,0.334939,0.329573,0.403310,0.340907,GALAXY,1
3,1.507000,1.512229,1.527126,1.591933,1.376655,1.435021,1.515309,1.514541,GALAXY,1
4,1.246063,1.230142,1.262818,0.992363,0.755588,0.917236,0.994422,0.883807,GALAXY,1


In [4]:
# Random Forest from Scikit-Learn: classify objects according to star (0) or galaxy (1)
# start with the usual train / test split
acc = 0
n_iter = 10
#state_list = [random.randrange(1, 9999, 1) for _ in range(10)] 
state_list = [3019, 2454, 4539, 2734, 9868, 2706, 2267, 9736, 4497, 659]

t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25, random_state = state_list[i])

    # define a model
    clf = RandomForestClassifier(n_estimators=50,n_jobs=3)
    
    # fit the model to the training data
    clf.fit(X_train, y_train)

    # predict object classes for the test data 
    preds_bin = clf.predict(X_test)
    # evaluate accuracy:    
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
acc_rf = acc
t_rf = t1-t0
# average accuracy over n_iter runs
print(type(clf).__name__)
print('average accuracy over '+str(n_iter)+' runs:',np.round(acc,4),' time elaped:',np.round(t1-t0,2),'s')

100%|██████████| 10/10 [00:18<00:00,  1.81s/it]

RandomForestClassifier
average accuracy over 10 runs: 0.9685  time elaped: 18.09 s


In [5]:
# defining useful function
def eval_results():
    if (acc - acc_rf) >= 0: plus = '+'
    else: plus = ''
    acc_diff_str = '('+plus+str(np.round(acc - acc_rf,4))+').'
    t_diff_str = '('+str(int((t1-t0)/t_rf*100.))+'%)'
    print(type(clf).__name__)
    print('average accuracy over '+str(n_iter)+' runs:',np.round(acc,4),acc_diff_str+' time elaped:',np.round(t1-t0,2),'s',t_diff_str)


In [6]:
# now lets try Catboost (for readability I refrain from defining classes or functions when repeating code blocks)

acc = 0
n_iter = 10
t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = state_list[i])
    clf = CatBoostClassifier(logging_level='Silent',thread_count=3, n_estimators=50)
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
eval_results()

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]

CatBoostClassifier
average accuracy over 10 runs: 0.9691 (+0.0006). time elaped: 7.25 s (40%)


In [7]:
# now lets try Light GBM

acc = 0
n_iter = 10
t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = state_list[i])
    clf = lgb.LGBMClassifier(n_jobs = 3,n_estimators=50)
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
eval_results()

100%|██████████| 10/10 [00:04<00:00,  2.20it/s]

LGBMClassifier
average accuracy over 10 runs: 0.9702 (+0.0018). time elaped: 4.57 s (25%)


In [8]:
# XGBoost (not ideal for this problem, but included for completeness)

acc = 0
n_iter = 10
t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = state_list[i])
    clf = xgb.XGBClassifier(n_estimators=50,n_jobs=3,tree_method='exact')
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)  
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
eval_results()

100%|██████████| 10/10 [00:11<00:00,  1.16s/it]

XGBClassifier
average accuracy over 10 runs: 0.9636 (-0.0048). time elaped: 11.63 s (64%)


In [9]:
#Scikit-Learn ExtraTreesClassifier

acc = 0
n_iter = 10
t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = state_list[i])
    clf = ExtraTreesClassifier(n_estimators=50,n_jobs=3)
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)  
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
eval_results()

100%|██████████| 10/10 [00:08<00:00,  1.15it/s]

ExtraTreesClassifier
average accuracy over 10 runs: 0.9682 (-0.0003). time elaped: 8.69 s (48%)


In [10]:
#GradientBoostingClassifier
# (apparently doesn't have an n_jobs parameter --> slow)

acc = 0
n_iter = 10
t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = state_list[i])
    clf = GradientBoostingClassifier(n_estimators=50)
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)  
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
eval_results()

100%|██████████| 10/10 [00:48<00:00,  4.84s/it]

GradientBoostingClassifier
average accuracy over 10 runs: 0.9634 (-0.005). time elaped: 48.42 s (267%)


In [11]:
# a simple deep learning NN from Scikit-Learn
# (slow but quite accurate)

acc = 0
n_iter = 10
t0 = time.time()
for i in tqdm(range(0,n_iter)):
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = state_list[i])
    clf = neural_network.MLPClassifier(activation='tanh', solver='adam', tol=1e-05, hidden_layer_sizes=(8,8,8,8),
                                       max_iter=300,learning_rate_init = 0.0001)
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    acc += accuracy_score(y_test,preds_bin) / n_iter

t1 = time.time()
eval_results()

100%|██████████| 10/10 [06:15<00:00, 37.53s/it]

MLPClassifier
average accuracy over 10 runs: 0.9704 (+0.0019). time elaped: 375.3 s (2074%)


In [12]:
# now let's look at simple model emsembling
# train different learners on the same data and combine the results

# train test split again
X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.25,random_state = 42)

# create a dictionary of learning algorithms
models = {'nn':neural_network.MLPClassifier(activation='tanh', solver='adam', tol=1e-05, 
                                            hidden_layer_sizes=(8,8,8,8),max_iter=300,
                                            learning_rate_init = 0.0001),
          'RandomForest':RandomForestClassifier(n_estimators=50,n_jobs=3),
          'lgbm':lgb.LGBMClassifier(n_jobs = 3,n_estimators=50),
          'catboost':CatBoostClassifier(logging_level='Silent',thread_count=3, n_estimators=50),
          'knn':neighbors.KNeighborsClassifier(n_neighbors=25),
          #'ExtraTrees':ExtraTreesClassifier(n_estimators=50,n_jobs=3),
          #'XGBoost':xgb.XGBClassifier(n_estimators=50,n_jobs=3,tree_method='exact')
         }

# set up empty arrays to compute average and hard votes
preds_array = np.zeros((len(models),len(y_test)))
preds_avg = np.zeros(len(y_test))

for i,name in enumerate(list(models.keys())):
    if i==0: 
        print('running models ...') 
    #train and predict
    clf = models[name]
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    preds_proba = clf.predict_proba(X_test)[:,1]
    
    # fill arrays
    preds_array[i,:] = preds_bin
    preds_avg += preds_proba / len(models)
    
    # accuracy of individual learners
    acc = accuracy_score(y_test,preds_bin)
    print(type(clf).__name__,np.round(acc,4))

preds_vote = np.where(np.mean(preds_array,axis=0) > 0.5, 1, 0) 

# check accuracy of ensembles
print('')
print('averaged predictions accuracy:',accuracy_score(y_test,np.where(preds_avg > 0.5, 1, 0)))
print('hard voting accuracy:',accuracy_score(y_test,preds_vote))


running models ...
MLPClassifier 0.9748
RandomForestClassifier 0.9738
LGBMClassifier 0.9748
CatBoostClassifier 0.975
KNeighborsClassifier 0.9736

averaged predictions accuracy: 0.976
hard voting accuracy: 0.9762


In [13]:
# now let's look at simple model emsembling
# train different learners on the same data and combine the results

df2 = df.copy()

#for col in cols:
#    df2[col] = df[col].sample(frac=0.99)
#df2.fillna(-99.9,inplace=True)

#df2['Target'] = df2['Target'].sample(frac=0.7)
#df2['Target'].fillna(0,inplace=True)
#df2['Target'] = df2['Target'].sample(frac=0.7)
#df2['Target'].fillna(1,inplace=True)
#print(df2.head())

# train test split again
X_train, X_test, y_train, y_test = train_test_split(df2[cols], df2['Target'],test_size=0.25,random_state = 42)

# create a dictionary of learning algorithms
models = {'RandomForest':RandomForestClassifier(n_estimators=50,n_jobs=3),
          'lgbm':lgb.LGBMClassifier(n_jobs = 3,n_estimators=50),
          'catboost':CatBoostClassifier(logging_level='Silent',thread_count=3, n_estimators=50),
          'knn':neighbors.KNeighborsClassifier(n_neighbors=25),
          'nn':neural_network.MLPClassifier(activation='tanh', solver='adam', tol=1e-05, 
                                            hidden_layer_sizes=(8,8,8,8),max_iter=300,
                                            learning_rate_init = 0.0001),
         }

# set up empty arrays to compute average and hard votes
preds_array = np.zeros((len(models),len(y_test)))
preds_avg = np.zeros(len(y_test))

for i,name in enumerate(list(models.keys())):
    if i==0: 
        print('running models ...') 
    #train and predict
    clf = models[name]
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    preds_proba = clf.predict_proba(X_test)[:,1]
    
    # fill arrays
    preds_array[i,:] = preds_bin
    preds_avg += preds_proba / len(models)
    
    # accuracy of individual learners
    acc = accuracy_score(y_test,preds_bin)
    print(type(clf).__name__,np.round(acc,4))

preds_vote = np.where(np.mean(preds_array,axis=0) > 0.5, 1, 0) 

# check accuracy of ensembles
print('')
print('averaged predictions accuracy:',accuracy_score(y_test,np.where(preds_avg > 0.5, 1, 0)))
print('hard voting accuracy:',accuracy_score(y_test,preds_vote))

running models ...
RandomForestClassifier 0.974
LGBMClassifier 0.9748
CatBoostClassifier 0.975
KNeighborsClassifier 0.9736
MLPClassifier 0.975

averaged predictions accuracy: 0.9764
hard voting accuracy: 0.976


In [14]:
# Generalised Stacking



In [27]:
# pseudo labeling: an advanced semi-supervised method
# let's apply this method using catboost 

print('individual learners:')
X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.975)
# evaluate accuracy of individual learners
acc_best = 0
for model in models.keys():
    clf = models[model]
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    acc = accuracy_score(y_test,preds_bin)
    print('accuracy score:',np.round(acc,4),'('+str(type(clf).__name__)+')')

# choose a learner for initial model:
#clf = models['nn']
clf = models['lgbm']
clf.fit(X_train, y_train)
preds_bin = clf.predict(X_test)
acc = accuracy_score(y_test,preds_bin)
print('')
print('pseudo-labeling loop:')
print('starting model accuracy:',np.round(acc,4),'('+str(type(clf).__name__)+')')
    
# create new training set by merging train and test data
X_train_p = pd.concat([X_train,X_test])

t0 = time.time()
for i in range(0,300):
    # generate new targets by merging train targets and test predictions (i.e., pseudo labels)
    y_train_p = y_train.tolist() + preds_bin.tolist()
    
    # alternate between tree and deep learning models
    if i % 2 != 0: model = 'nn'
    else: model = 'lgbm'
    
    clf = models[model]
    clf.fit(X_train_p, y_train_p)
    preds_bin = clf.predict(X_test)
    acc = accuracy_score(y_test,preds_bin)
    t1 = time.time()
    print(i+1,'pseudo labeling accuracy score:',np.round(acc,4),'('+str(type(clf).__name__)+')')

t1 = time.time()
print('done: ('+str(np.round(t1-t0,1))+') s')

individual learners:
accuracy score: 0.9384 (RandomForestClassifier)
accuracy score: 0.9268 (LGBMClassifier)
accuracy score: 0.9221 (CatBoostClassifier)
accuracy score: 0.9154 (KNeighborsClassifier)
accuracy score: 0.9475 (MLPClassifier)

pseudo-labeling loop:
starting model accuracy: 0.9268 (LGBMClassifier)
1 pseudo labeling accuracy score: 0.9275 (LGBMClassifier)
2 pseudo labeling accuracy score: 0.9482 (MLPClassifier)
3 pseudo labeling accuracy score: 0.9489 (LGBMClassifier)
4 pseudo labeling accuracy score: 0.9551 (MLPClassifier)
5 pseudo labeling accuracy score: 0.9556 (LGBMClassifier)
6 pseudo labeling accuracy score: 0.9561 (MLPClassifier)
7 pseudo labeling accuracy score: 0.9567 (LGBMClassifier)
8 pseudo labeling accuracy score: 0.9584 (MLPClassifier)
9 pseudo labeling accuracy score: 0.9587 (LGBMClassifier)
10 pseudo labeling accuracy score: 0.959 (MLPClassifier)
11 pseudo labeling accuracy score: 0.9596 (LGBMClassifier)
12 pseudo labeling accuracy score: 0.9596 (MLPClassifier

136 pseudo labeling accuracy score: 0.9697 (MLPClassifier)
137 pseudo labeling accuracy score: 0.9699 (LGBMClassifier)
138 pseudo labeling accuracy score: 0.9699 (MLPClassifier)
139 pseudo labeling accuracy score: 0.9697 (LGBMClassifier)
140 pseudo labeling accuracy score: 0.9701 (MLPClassifier)
141 pseudo labeling accuracy score: 0.9699 (LGBMClassifier)
142 pseudo labeling accuracy score: 0.9696 (MLPClassifier)
143 pseudo labeling accuracy score: 0.9695 (LGBMClassifier)
144 pseudo labeling accuracy score: 0.9699 (MLPClassifier)
145 pseudo labeling accuracy score: 0.97 (LGBMClassifier)
146 pseudo labeling accuracy score: 0.9699 (MLPClassifier)
147 pseudo labeling accuracy score: 0.9698 (LGBMClassifier)
148 pseudo labeling accuracy score: 0.9699 (MLPClassifier)
149 pseudo labeling accuracy score: 0.9698 (LGBMClassifier)
150 pseudo labeling accuracy score: 0.9698 (MLPClassifier)
151 pseudo labeling accuracy score: 0.97 (LGBMClassifier)
152 pseudo labeling accuracy score: 0.9699 (MLPClass

275 pseudo labeling accuracy score: 0.9697 (LGBMClassifier)
276 pseudo labeling accuracy score: 0.9703 (MLPClassifier)
277 pseudo labeling accuracy score: 0.97 (LGBMClassifier)
278 pseudo labeling accuracy score: 0.9699 (MLPClassifier)
279 pseudo labeling accuracy score: 0.9699 (LGBMClassifier)
280 pseudo labeling accuracy score: 0.9704 (MLPClassifier)
281 pseudo labeling accuracy score: 0.9704 (LGBMClassifier)
282 pseudo labeling accuracy score: 0.9707 (MLPClassifier)
283 pseudo labeling accuracy score: 0.9706 (LGBMClassifier)
284 pseudo labeling accuracy score: 0.9706 (MLPClassifier)
285 pseudo labeling accuracy score: 0.9705 (LGBMClassifier)
286 pseudo labeling accuracy score: 0.9705 (MLPClassifier)
287 pseudo labeling accuracy score: 0.9704 (LGBMClassifier)
288 pseudo labeling accuracy score: 0.9705 (MLPClassifier)
289 pseudo labeling accuracy score: 0.9704 (LGBMClassifier)
290 pseudo labeling accuracy score: 0.9708 (MLPClassifier)
291 pseudo labeling accuracy score: 0.9705 (LGBMCl

In [26]:
# pseudo labeling: same thing, smaller test data size

print('individual learners:')
X_train, X_test, y_train, y_test = train_test_split(df[cols], df['Target'],test_size=0.33)
# evaluate accuracy of individual learners
acc_best = 0
for model in models.keys():
    clf = models[model]
    clf.fit(X_train, y_train)
    preds_bin = clf.predict(X_test)
    acc = accuracy_score(y_test,preds_bin)  
    print('accuracy score:',np.round(acc,4),'('+str(type(clf).__name__)+')')

# choose a learner for initial model:
#clf = models['nn']
clf = models['lgbm']
clf.fit(X_train, y_train)
preds_bin = clf.predict(X_test)
acc = accuracy_score(y_test,preds_bin)
print('')
print('pseudo-labeling loop:')
print('starting model accuracy:',np.round(acc,4),'('+str(type(clf).__name__)+')')
    
# create new training set by merging train and test data
X_train_p = pd.concat([X_train,X_test])

t0 = time.time()
for i in range(0,3):
    # generate new targets by merging train targets and test predictions (i.e., pseudo labels)
    y_train_p = y_train.tolist() + preds_bin.tolist()
    
    # alternate between tree and deep learning models
    if i % 2 == 0: model = 'nn'
    else: model = 'lgbm'
    
    clf = models[model]
    clf.fit(X_train_p, y_train_p)
    preds_bin = clf.predict(X_test)
    acc = accuracy_score(y_test,preds_bin)
    t1 = time.time()
    print(i+1,'pseudo labeling accuracy score:',np.round(acc,4),'('+str(type(clf).__name__)+')')

t1 = time.time()
print('done: ('+str(np.round(t1-t0,1))+') s')

individual learners:
accuracy score: 0.9718 (RandomForestClassifier)
accuracy score: 0.9739 (LGBMClassifier)
accuracy score: 0.973 (CatBoostClassifier)
accuracy score: 0.9717 (KNeighborsClassifier)
accuracy score: 0.9727 (MLPClassifier)

pseudo-labeling loop:
starting model accuracy: 0.9739 (LGBMClassifier)
1 pseudo labeling accuracy score: 0.9735 (MLPClassifier)
2 pseudo labeling accuracy score: 0.9735 (LGBMClassifier)
3 pseudo labeling accuracy score: 0.9729 (MLPClassifier)
done: (96.6) s
